# Environment setup

You can setup the workspace of the application with:
```shell
pip install requirements.txt
```

In [1]:
# lisa
from fastgb.fastgb import FastGB
import lisaorbits
import lisaconstants

# display module
import plotly.graph_objects as go

# common
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline as spline
import os.path

# homemade
from appFOM import LISA_GB_configuration as myGB
from appFOM import LISA_noise_configuration as NOISE
from appFOM import utils

# jupyter widgets
import ipywidgets as widgets
from ipywidgets import Layout

# Sensitivity plot

## Data generation

In [2]:
# TO DO

## Plot creation and display

### Data initialisation

In [3]:
### Data download

if not os.path.exists("VGB.npy"):
    !wget https://github.com/Salander619/AppStreamlit/raw/main/data/VGB.npy

--2024-03-18 14:29:59--  https://github.com/Salander619/AppStreamlit/raw/main/data/VGB.npy
Résolution de github.com (github.com)… 140.82.121.4
Connexion à github.com (github.com)|140.82.121.4|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://raw.githubusercontent.com/Salander619/AppStreamlit/main/data/VGB.npy [suivant]
--2024-03-18 14:29:59--  https://raw.githubusercontent.com/Salander619/AppStreamlit/main/data/VGB.npy
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 9592 (9,4K) [application/octet-stream]
Enregistre : ‘VGB.npy’

VGB.npy             100%[===================>]   9,37K  --.-KB/s    ds 0s      

2024-03-18 14:29:59 (20,4 MB/s) - ‘VGB.npy’ enregistré [9592/9592]



In [4]:
### data load
# verification GB reader
input_gb_filename = "VGB.npy"

gb_config_file = np.load(input_gb_filename)
nb_of_sources = len(gb_config_file)
GB_out = np.rec.fromarrays(
    [
        np.zeros((nb_of_sources, 1)),
        np.zeros((nb_of_sources, 1)),
        np.zeros((nb_of_sources, 1)),
    ],
    names=["freq", "sh", "snr"],
)

list_of_names = gb_config_file["Name"]

list_of_names_opt = list_of_names
list_of_names_opt = np.append("select all", list_of_names_opt)

list_of_sources = []
list_of_amplitude = []

### Data preparation

In [5]:
# widget variables
list_of_GB = []
mission_duration = 5.0
tdi2 = True
display_mode = "x unified"

In [6]:

####### prepare the data
# noise
test0 = NOISE.LISA_analytical_noise("dummy", 42)

freq = np.logspace(-5, 0, 9990)
duration = mission_duration  # years
tobs = (
    duration
    * lisaconstants.SIDEREALYEAR_J2000DAY  # pylint: disable=no-member
    * 24
    * 60
    * 60210
)
lisa_orbits = lisaorbits.EqualArmlengthOrbits(
    dt=8640,
    size=(tobs + 10000) // 8640,
)
# to control the +10000

# noise psd
SXX_noise_instru_only = test0.instru_noise_psd(freq, tdi2_=tdi2, option_="X")
SXX_confusion_noise_only = test0.confusion_noise_psd(
    freq, duration_=duration, tdi2_=tdi2, option_="X"
)
SXX_noise = SXX_noise_instru_only + SXX_confusion_noise_only

# response
R_ = utils.fast_response(freq, tdi2=tdi2)
R = spline(freq, R_)
# NOISE SENSITIVITY
sh = spline(freq, SXX_noise_instru_only / R_)
sh_wd = utils.psd2sh(freq, SXX_noise, sky_averaging=False, tdi2=tdi2)

# signal

GB = FastGB(delta_t=15, T=tobs, orbits=lisa_orbits, N=1024)
df = 1 / tobs


for j, s in enumerate(gb_config_file):

    pGW = dict(zip(gb_config_file.dtype.names, s))

    if pGW["Name"] in list_of_GB:

        params = np.array(
            [
                pGW["Frequency"],
                pGW["FrequencyDerivative"],
                pGW["Amplitude"],
                pGW["EclipticLatitude"],
                pGW["EclipticLongitude"],
                pGW["Polarization"],
                pGW["Inclination"],
                pGW["InitialPhase"],
            ]
        )

        source_tmp = myGB.LISA_GB_source(pGW["Name"], params)
        list_of_sources.append(source_tmp)
        list_of_amplitude.append(
            source_tmp.get_source_parameters()[0][2] / (1e-23),
        )

        X, _, _, kmin = GB.get_fd_tdixyz(
            source_tmp.get_source_parameters(),
            tdi2=True,
        )
        X_f = df * np.arange(kmin, kmin + len(X.flatten()))

        h0 = np.sqrt(4 * df * float(np.sum(np.abs(X) ** 2 / R(X_f))))
        h0 *= np.sqrt(2)
        GB_out["sh"][j] = h0**2
        GB_out["freq"][j] = pGW["Frequency"]

### Display of the curves

In [7]:
####### display the sensitivity curves
vf = []
vy = []

for vgb in GB_out:
    vf.append(float(vgb["freq"]))
    vy.append(float(np.sqrt(vgb["freq"] * vgb["sh"])))

## end of fake data

fig = go.Figure()

tmp = list_of_names.tolist()  # list_of_GB

fig.add_trace(
    go.Scatter(
        x=vf,
        y=vy,
        hovertext=tmp,
        # visible='legendonly',
        mode="markers",
        marker={"color": "red"},
        marker_symbol="hexagon",
        name="GBs",
        hovertemplate="<b>%{hovertext}</b><br>f= %{x:.4f} Hz<br>h=%{y}",
    )
)

fig.add_trace(
    go.Scatter(
        x=freq,
        y=np.sqrt(freq) * np.sqrt(sh(freq)),
        name="Instrumental Noise",
    )
)


fig.add_trace(
    go.Scatter(
        x=freq,
        y=np.sqrt(freq) * np.sqrt(20 / 3) * np.sqrt(sh_wd(freq)),
        name="LISA Noise (Instru+Confusion)",
    )
)


fig.update_xaxes(
    title_text="Frequency (Hz)",
    type="log",
    showgrid=True,
    showexponent="all",
    exponentformat="e",
)
fig.update_yaxes(
    title_text="Characteristic Strain (TODO)",
    type="log",
    showgrid=True,
)
fig.update_layout(xaxis={"range": [-5, 0]})
fig.update_layout(yaxis={"range": [-22, -15]})
fig.update_layout(template="ggplot2")

fig.update_layout(hovermode=display_mode)

fig.update_layout(
    legend={
        "orientation": "h",
        "yanchor": "bottom",
        "y": 1.02,
        "xanchor": "right",
        "x": 1,
    }
)

fig.update_layout(height=600, width=1000)
fig.show()

In [8]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=freq, y=SXX_noise_instru_only, name="instru"))

fig2.add_trace(
    go.Scatter(
        x=freq,
        y=SXX_confusion_noise_only,
        # visible='legendonly',
        name="confusion",
    )
)

fig2.update_xaxes(
    title_text="Frequency (Hz)",
    type="log",
    showgrid=True,
    showexponent="all",
    exponentformat="e",
)
fig2.update_yaxes(
    title_text="Characteristic Strain (TODO)",
    type="log",
    showgrid=True,
)
fig2.show()